# Drizzle 2: Bayer drizzle

Drizzle separately each of the R, G, and B arrays of a Bayer image. 

At the end, combine the resulting 3 arrays as a 3-extension FITS file. 

It is assujed the input was already background-subtracted.

Originally this notebook was developed with the ISO 12800 data set as the test data. In the current version, it is configured to use the ISO 6400 data set. Statements specific to the original data set were comented out.

In [1]:
# %matplotlib ipympl

import os, glob
import datetime

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.visualization import make_lupton_rgb
from drizzle import cdrizzle
import rawpy

from datapath import DATA

# for background subtraction experiment
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans
from astropy.stats import SigmaClip
from photutils import Background2D, MedianBackground, ModeEstimatorBackground

/var/folders/y5/zcwj7zc17g19nggvb1vx4rc80000gn/T/ipykernel_32625/3982284005.py:20: DeprecationWarning: `photutils.Background2D` is a deprecated alias for `photutils.background.Background2D` and will be removed in the future. Instead, please use `from photutils.background import Background2D` to silence this warning.
  from photutils import Background2D, MedianBackground, ModeEstimatorBackground
/var/folders/y5/zcwj7zc17g19nggvb1vx4rc80000gn/T/ipykernel_32625/3982284005.py:20: DeprecationWarning: `photutils.MedianBackground` is a deprecated alias for `photutils.background.MedianBackground` and will be removed in the future. Instead, please use `from photutils.background import MedianBackground` to silence this warning.
  from photutils import Background2D, MedianBackground, ModeEstimatorBackground
/var/folders/y5/zcwj7zc17g19nggvb1vx4rc80000gn/T/ipykernel_32625/3982284005.py:20: DeprecationWarning: `photutils.ModeEstimatorBackground` is a deprecated alias for `photutils.background.ModeE

## Functions

In [2]:
def drizz_rgb(imarray, pixmap, output_array, weights, output_counts, output_context, 
              aslice=None):
    # for diagnostic purposes, this function will also return stats associated 
    # with each one of the input RGB arrays
    stats = []
    
    # loop over the RGB bands    
    for band in list(range(3)):
        
        # generate zeroed array and add the input rawpy array to it. This is necessary 
        # since calling cdrizzle.tdriz directly with the rawpy-generated array as input,
        # causes an exception to be raised by cdrizzle. 
        # 
        # We also need to apply the mask/weights here, because drizzle will re-scale 
        # the mask when doing pixel re-scaling.
        imarray_copy = np.zeros(imarray.shape, dtype='float32')
        imarray_copy += imarray * weights[band]

        # stats for diagnostics
        imarray_stats = imarray_copy
        imarray_stats[imarray_copy == 0.] = np.NaN
        stats.append(np.nanmean(imarray_stats[aslice]))
        
        # call C core drizzle algorithm
        cdrizzle.tdriz(imarray_copy, weights[band], pixmap, output_array[band], 
                       output_counts[band], output_context[band], 
                       kernel="turbo", expscale=1.)
        
    return stats

In [3]:
def drizz(filename, output_array, weights, output_counts, output_context):
    '''
    Drizzle one raw image. 
    
    This works by breaking up the raw array into each one of its 3 sub-images,
    R, G, and B, and drizzling them separately into their own respective
    output arrays. A single pixel mapping file containing the offsets associated 
    with the input raw image is used. This is the file created by the Offsets_X 
    scripts.
    
    The outputs are 3-tuples that store respectively the R, G, and B results 
    in each one of their elements. They have to be allocated and initialized
    by the caller.
    
    The weights array must have the RGB pixel masks multiplied into them.
    '''
    # read input
#     raw = rawpy.imread(filename)
#     imarray = raw.raw_image_visible.astype(float)

    f = fits.open(filename)
    imarray = f[1].data
    f.close()   
    
    # read offsets
    fname_offsets = filename.replace('cutout.fits', 'offsets.fits')
    f = fits.open(fname_offsets)
    offsets_x = f[1].data
    offsets_y = f[2].data
    f.close()
    
    # build 1-to-1 pixmap
    idxmap = np.indices((imarray.shape[1], imarray.shape[0]), dtype='float64')

    # add offsets
    idxmap[0] -= offsets_x.transpose()
    idxmap[1] -= offsets_y.transpose()

    # reshape for drizzle
    idxmap = idxmap.transpose()
    idxmap = idxmap.reshape(imarray.shape[1] * imarray.shape[0], 2)
    pixmap = idxmap.reshape(imarray.shape[0], imarray.shape[1], 2)
    
    # call drizzle on the RGB arrays
#     drizz_rgb(subtracted, pixmap, output_array, weights, output_counts, output_context)
    stats = drizz_rgb(imarray, pixmap, output_array, weights, output_counts, output_context,
                      aslice=np.index_exp[700:800,1250:1300])
    
    # print stats
    _, fname = os.path.split(fpath)
    print(fname, stats)

## Read reference image

In [4]:
# reference image used in the Offsets_X scripts. Offsets are defined against this image.
# This reference is used for both the ISO 12800 and 6400 data sets.
# datadir = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/12800/light')
# reference_fname = os.path.join(datadir,'DSC03770.ARW')

# reference for the Andromeda_2022 data set
datadir = os.path.join(DATA,'astrophotography_data/Andromeda_2022/135mm16s3200ISO/')
reference_fname = os.path.join(datadir,'DSC01263.cutout.fits')
reference_fname_arw = os.path.join(datadir,'DSC01263.ARW')

f = fits.open(reference_fname)
reference_imarray = f[1].data
f.close()    

raw = rawpy.imread(reference_fname_arw)

## Build masks that isolate the R, G, and B layers from a Bayer array

This step must run only after an actual image was read by rawpy, so the Bayer encoding of said image becomes available.

In [5]:
# Bayer code mask should be the same size as cutout
yr0 = 0
yr1 = reference_imarray.shape[0]
xr0 = 0
xr1 = reference_imarray.shape[1]

bayer_slice = np.index_exp[yr0:yr1,xr0:xr1]

In [6]:
colors_array = raw.raw_colors_visible
# mask = np.ones(shape=colors_array.shape, dtype='float32')

red_mask = np.where(colors_array == 0, 1, 0).astype(np.float32)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = (green_mask_1 | green_mask_2).astype(np.float32)

blue_mask = np.where(colors_array == 2, 1, 0).astype(np.float32)

masks = []
masks.append(red_mask[bayer_slice])
masks.append(green_mask[bayer_slice])
masks.append(blue_mask[bayer_slice])

## Parameters to control background subtraction

In [7]:
bkg_cell_footprint = (25, 25)
bkg_filter = (9, 9)

bkg_sigma_clip = SigmaClip(sigma=5.)
bkg_kernel = Gaussian2DKernel(x_stddev=3)
bkg_estimator = ModeEstimatorBackground()

## Define list of images to be processed

In [8]:
# path1 = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/12800/light')   # ISO 12800 data set
# path2 = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/6400/light')   # ISO 6400 data set

# list_p_1 = glob.glob(path1 + '/*.ARW')[1:]
# list_p_2 = glob.glob(path2 + '/*.ARW')
# list_p = list_p_2[20:] + list_p_1

# list_r = [reference_fname]

# list_p.sort()
# list_p.reverse()
# list_p = list_r + list_p


# Andromeda_2022 data set
list_p = glob.glob(datadir + '/*.cutout.fits')
list_p.sort()
list_p = list_p[1:]

## Allocate and initialize arrays used by drizzle

Use Bayer masks as weights for drizzling.

In [9]:
weights = []
output_array = []
output_counts = []
output_context = []

for band in list(range(3)):

    weights.append(np.ones(reference_imarray.shape, dtype='float32') * masks[band])

    output_array.append(np.zeros(reference_imarray.shape, dtype='float32'))
    output_counts.append(np.zeros(reference_imarray.shape, dtype='float32'))
    output_context.append(np.zeros(reference_imarray.shape, dtype='int32'))

In [10]:
# print(weights[0])
# print()
# print(weights[1])
# print()
# print(weights[2])

## Loop over list of images

In [ ]:
for fpath in list_p:
    drizz(fpath, output_array, weights, output_counts, output_context)

DSC01263.cutout.fits [2.0549164, 5.136837, 1.8887959]
DSC01264.cutout.fits [2.7879403, 4.793776, 0.89654475]
DSC01265.cutout.fits [3.2025754, 4.8711596, 2.555418]
DSC01266.cutout.fits [3.2845926, 5.4212923, 2.4021282]
DSC01267.cutout.fits [3.2708225, 5.4147186, 1.8653097]
DSC01268.cutout.fits [3.299307, 4.5147576, 1.5724821]
DSC01269.cutout.fits [2.3234723, 4.885042, 2.6000423]
DSC01270.cutout.fits [3.4455347, 5.06667, 2.872842]
DSC01271.cutout.fits [2.8114734, 4.5708337, 2.488187]
DSC01272.cutout.fits [3.2330277, 4.3828983, 1.0689238]
DSC01273.cutout.fits [2.6318495, 4.79164, 1.8020886]
DSC01274.cutout.fits [3.4837391, 5.9091983, 3.6514657]
DSC01275.cutout.fits [2.936111, 5.619359, 4.6849203]
DSC01276.cutout.fits [2.8652055, 4.9114537, 3.318901]
DSC01277.cutout.fits [4.198872, 4.2596326, 1.9776623]
DSC01278.cutout.fits [2.6580992, 4.1756406, 4.489646]
DSC01279.cutout.fits [3.6397219, 5.7011423, 2.0257246]
DSC01280.cutout.fits [3.941625, 5.3315663, 2.15578]
DSC01281.cutout.fits [2.2452

DSC01415.cutout.fits [3.5284, 6.6148305, 3.5909054]
DSC01416.cutout.fits [2.5335667, 6.809967, 3.987782]
DSC01417.cutout.fits [3.097857, 6.5825553, 3.8915648]
DSC01418.cutout.fits [2.7234216, 7.6803236, 3.8276587]
DSC01419.cutout.fits [4.466815, 6.9214506, 5.2571383]
DSC01420.cutout.fits [4.489166, 7.6132345, 3.70919]
DSC01421.cutout.fits [4.9366355, 7.0178127, 3.551392]
DSC01422.cutout.fits [3.4778879, 7.6492195, 5.166875]
DSC01423.cutout.fits [3.9978702, 7.5765533, 4.2114825]
DSC01424.cutout.fits [4.8818164, 5.4927945, 4.494902]
DSC01425.cutout.fits [3.4397767, 6.711228, 4.4366226]
DSC01426.cutout.fits [3.5151453, 8.078708, 4.537682]
DSC01427.cutout.fits [2.5788817, 6.843208, 3.5502234]
DSC01428.cutout.fits [4.4134974, 7.015012, 3.8293483]
DSC01429.cutout.fits [3.3204691, 6.9645023, 5.0219307]
DSC01430.cutout.fits [5.131989, 7.1513267, 2.6378622]
DSC01431.cutout.fits [3.7258134, 7.220193, 3.4897351]
DSC01432.cutout.fits [4.908928, 6.1258373, 4.363014]
DSC01433.cutout.fits [4.904949, 

DSC01568.cutout.fits [8.217469, 15.673111, 10.257179]
DSC01569.cutout.fits [6.989992, 14.442373, 10.018283]
DSC01570.cutout.fits [8.130938, 14.495668, 10.479796]
DSC01571.cutout.fits [6.431381, 14.109114, 11.413648]
DSC01572.cutout.fits [7.417986, 15.1079235, 10.875922]
DSC01573.cutout.fits [6.114745, 14.080521, 10.041425]
DSC01574.cutout.fits [6.975619, 14.290685, 10.187961]
DSC01575.cutout.fits [7.0184436, 13.257756, 9.765476]
DSC01576.cutout.fits [6.2940288, 14.145909, 9.438531]
DSC01577.cutout.fits [7.738378, 13.015926, 10.462795]
DSC01578.cutout.fits [7.269064, 13.949208, 12.256445]
DSC01579.cutout.fits [6.269082, 13.51378, 12.705097]
DSC01580.cutout.fits [6.181177, 13.11837, 11.191438]
DSC01581.cutout.fits [7.3522677, 12.805604, 9.434063]
DSC01582.cutout.fits [8.305314, 13.967297, 10.198806]
DSC01583.cutout.fits [7.3532424, 14.8021, 12.967072]
DSC01584.cutout.fits [7.4824357, 14.299462, 10.365552]
DSC01585.cutout.fits [5.858155, 14.066084, 9.988313]
DSC01586.cutout.fits [7.510574

DSC01721.cutout.fits [6.2183547, 12.578008, 10.522736]
DSC01722.cutout.fits [6.7439275, 12.626024, 9.983594]
DSC01723.cutout.fits [5.543916, 12.252744, 11.902921]
DSC01724.cutout.fits [6.988275, 13.870994, 9.071681]
DSC01725.cutout.fits [5.43969, 12.845481, 10.547426]
DSC01726.cutout.fits [6.0752244, 12.453301, 9.65411]
DSC01727.cutout.fits [5.9681187, 11.871117, 10.5388155]
DSC01728.cutout.fits [7.1031084, 12.03881, 9.449407]
DSC01729.cutout.fits [5.8430657, 14.069714, 7.4970303]
DSC01730.cutout.fits [5.991694, 13.004569, 9.882102]
DSC01731.cutout.fits [6.686461, 13.864735, 9.534838]
DSC01732.cutout.fits [6.458268, 12.850342, 9.716669]
DSC01733.cutout.fits [7.208868, 13.587327, 9.983534]
DSC01734.cutout.fits [6.026688, 12.523676, 8.223628]
DSC01735.cutout.fits [6.577908, 13.383956, 8.478073]
DSC01736.cutout.fits [5.6880608, 14.021144, 7.2405305]
DSC01737.cutout.fits [6.6285233, 12.752763, 7.8716626]
DSC01738.cutout.fits [8.054729, 12.458045, 11.653497]
DSC01739.cutout.fits [6.1708846,

DSC01874.cutout.fits [3.1634731, 7.06722, 3.7809937]
DSC01875.cutout.fits [3.288266, 6.7031407, 3.1747508]
DSC01876.cutout.fits [2.9336712, 7.638738, 5.099952]
DSC01877.cutout.fits [3.0965226, 7.9904375, 3.9891617]
DSC01878.cutout.fits [2.7908762, 6.9760194, 6.9107227]
DSC01879.cutout.fits [3.7200484, 5.680899, 4.103427]
DSC01880.cutout.fits [3.5713868, 7.159295, 4.80181]
DSC01881.cutout.fits [4.6861835, 6.2331805, 5.8204436]
DSC01882.cutout.fits [3.5295398, 6.8007984, 5.875758]
DSC01883.cutout.fits [3.4847016, 6.969793, 3.7661688]
DSC01884.cutout.fits [3.428492, 7.034526, 4.372441]
DSC01885.cutout.fits [1.1674079, 6.9649916, 4.7488303]
DSC01886.cutout.fits [3.7611961, 6.262431, 6.743706]
DSC01887.cutout.fits [3.0334253, 6.679084, 4.5865765]
DSC01888.cutout.fits [3.8759527, 6.9789376, 5.106497]
DSC01889.cutout.fits [4.8413568, 8.042398, 4.056303]
DSC01890.cutout.fits [2.8261065, 7.503331, 4.3428955]
DSC01891.cutout.fits [3.910185, 7.266572, 3.5588005]
DSC01892.cutout.fits [3.3774257, 8

In [ ]:
coadded_r = output_array[0]
coadded_g = output_array[1]
coadded_b = output_array[2]

In [ ]:
# print(np.mean(coadded_r))
# print(np.mean(coadded_g))
# print(np.mean(coadded_b))

## Subtract background

In case it wasn't subtracted individually from each input image.

In [ ]:
# red_bkg = Background2D(coadded_r, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)
# green_bkg = Background2D(coadded_g, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)
# blue_bkg = Background2D(coadded_b, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)

# coadded_r -= red_bkg.background 
# coadded_g -= green_bkg.background
# coadded_b -= blue_bkg.background 

## Write FITS with result

In [ ]:
# Build 3-extension FITS file with R,G,B drizzled arrays in separate HDUs 
today = datetime.datetime.now().ctime()
drizz_name = os.path.join(datadir, 'drizzle.fits')
print(drizz_name, today)

hdr = fits.Header()
hdr['DATE'] = today
hdr['PATH'] = drizz_name
primary_hdu = fits.PrimaryHDU(header=hdr)

red_hdu   = fits.ImageHDU(coadded_r.astype('float32'))
green_hdu = fits.ImageHDU(coadded_g.astype('float32'))
blue_hdu  = fits.ImageHDU(coadded_b.astype('float32'))

red_hdu.header['BAND'] = 'RED'
green_hdu.header['BAND'] = 'GREEN'
blue_hdu.header['BAND'] = 'BLUE'

hdul = fits.HDUList([primary_hdu, red_hdu, green_hdu, blue_hdu])
hdul.writeto(drizz_name, overwrite=True)

## Plot diagnostics

In [ ]:
hdul = fits.open(drizz_name)
coadded_r = hdul[1].data
coadded_g = hdul[2].data
coadded_b = hdul[3].data

In [ ]:
# aslice = np.index_exp[1100:1290,1900:2100]
# aslice = np.index_exp[800:1800,2200:3200]
# aslice = np.index_exp[1150:1350,2250:2440]
# aslice = np.index_exp[1050:1250,2750:2950] # M31
aslice = np.index_exp[0:coadded_r.shape[0], 0:coadded_r.shape[1]] # Andromeda_2022
# aslice = np.index_exp[700:800,1250:1300] # Andromeda_2022

In [ ]:
vmax = 300

fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(2, 2, 1)
plt.imshow(coadded_g[aslice], vmin=0, vmax=vmax)
ax1 = fig.add_subplot(2, 2, 2, sharex=ax, sharey=ax)
plt.imshow(coadded_b[aslice], vmin=0, vmax=vmax)
ax2 = fig.add_subplot(2, 2, 3, sharex=ax, sharey=ax)
plt.imshow(coadded_r[aslice], vmin=0, vmax=vmax)

In [ ]:
fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(2, 2, 1)
plt.imshow(output_counts[1][aslice])
plt.colorbar()
ax1 = fig.add_subplot(2, 2, 2, sharex=ax, sharey=ax)
plt.imshow(output_counts[2][aslice])
plt.colorbar()
ax2 = fig.add_subplot(2, 2, 3, sharex=ax, sharey=ax)
plt.imshow(output_counts[0][aslice])
plt.colorbar()

In [ ]:
test_image = coadded_r * 1.93 + coadded_g + coadded_b * 1.90

fig = plt.figure(figsize=[12, 6])
ax = fig.add_subplot(1, 1, 1)
plt.imshow(test_image[aslice], vmin=0, vmax=600)

In [ ]:
rgb_image = make_lupton_rgb(coadded_r * 1.8, coadded_g, coadded_b * 1.8, Q=0.0001, stretch=80., minimum=-10.)

fig = plt.figure(figsize=[12, 6])
ax = fig.add_subplot(1, 1, 1)
plt.imshow(rgb_image[aslice])

In [ ]:
fig = plt.figure(figsize=[12, 6])
ax = fig.add_subplot(1, 1, 1)
plt.imshow(rgb_image)

In [ ]:
rgb_image.shape

In [ ]:
plt.imsave("drizzle.tiff", rgb_image)

In [ ]:
test = coadded_g - coadded_r

vmax = 25

fig = plt.figure(figsize=[12, 12])
ax = fig.add_subplot(1, 1, 1)
plt.imshow(test[aslice], vmin=0, vmax=vmax)